THIS IS AN EXAMPLE ABOUT HOW TO DOWNLOAD A DATABASE, CREATE A PICKLE FILE AND THEN LOAD IT INTO A NETWORK

In [ ]:
#first import pypath and the omnipath module

from pypath.core import network
from pypath.resources import data_formats as netres
from pypath import omnipath

In [ ]:
#then import the wrapper

from pypath_wrapper import Wrap_net

In [ ]:
w = Wrap_net()

#here we create the wrap_net object, which is the same as Network() but with a couple of functions more

In [ ]:
#here we load the database to populate the wrap_net object. It can be done in two ways:

#1) using the "netres" object
netres.omnipath

#in the output of the cells you can visualize the database that will populate your wrapnet object

In [ ]:
#to load the previous databses just do:

w.load(netres.pathway)

#the load methods, loads the databases contained in the netres object

In [ ]:
netres.pathway.keys()

In [ ]:
#2) an alternative is using the omnipath object

#the following lines will download the chosen databases, populate the cache folder and create a pickle file

#the first time this will take a lot of time because it will download Gb of data

#after the first time, it will take just a couple of seconds

omnipath.db.param['network_omnipath_mod'] = 'network' 

#use the following line if you want to specify some arguments and load just one or some databases (ex signor, kegg etc...)
omnipath.db.param['network_omnipath_args'] = {'resources': netres.pathway['signor']} # <-- insert in the dict the pathway

#get the actual files necessary to build the database

omnipath.db.get_db('network_omnipath')

In [ ]:
#next time when you want to launch the session rapidly you can just use:

m = omnipath.db.pickle_path('network_omnipath') #here you store the pickle file in the variable "m" in order to load it next

In [ ]:
#once the pickle file is created, let's load it into the new object

w = Wrap_net(m)

In [ ]:
#now let's check its dimensions
w

In [ ]:
#and let's have a look to the interactions:
w.interactions

SECOND PART: FROM A LIST OF GENES TO A BNET FILE

In [ ]:
#let's create a list of genes

genes = ["SRC", "NOTCH1", "FAK", "CDH1", "CDH2", "VIM", "MAP4K4", "LATS1", "LATS2"]

In [ ]:
from pypath.utils import mapping

for gene in genes:
    uniprot = list(mapping.map_name(gene, 'genesymbol', 'uniprot'))
    genesymbol = list(mapping.map_name(uniprot[0], 'uniprot', 'genesymbol'))
    print(uniprot, genesymbol)

In [ ]:
#and now let's extract a subnetwork from our wrap_net object that contains the interactions between those genes

#let's propose a max distance between each gene, in this case, let's say 1

distance = 2

new_net = w.extract_subnet(genes, distance, complete_connections=False, resources="SIGNOR")

In [ ]:
#let's check the new net

new_net.generate_graph()
new_net.print_graph()

In [ ]:
#the previous graph shows some disconnected nodes, this is normal because not every gene/protein is in direct
#connection with the other ones
#So now, let's use the "complete_connections" flag in 'soft' mode

#this flag will trigger a function that adds JUST the FIRST shortest path that can connect all the nodes

#let's call this network the "minimum connected network"

distance = 2

new_net = w.extract_subnet(genes, distance, complete_connections="soft", resources="SIGNOR")

In [ ]:
#let's check the new net

new_net.generate_graph()
new_net.print_graph()

In [ ]:
#the previous network already include a good number of genes

#but sometimes, when working on some biological context, those genes are not enough.

#this time we want to enrich even more the network to find any possible gene that could be implicated in our study

#to do this, let's use again the flag "complete_connections" setting it on True

#this will trigger a function that will look for a possible path between every couple of genes
#rising by a lot the number of final nodes in the network (WARNING: IT COULD TAKES A WHILE)

distance = 2

new_net = w.extract_subnet(genes, distance, complete_connections=True, resources="SIGNOR")

In [ ]:
#let's check the new net

new_net.generate_graph()
new_net.print_graph()

In [ ]:
#Now, let's say that I have a big network and I need all the connected genes to a specific one, here is how to 
# retrieve them:

a = new_net.get_neighbours("MAP4K4")
print(a)

In [ ]:
# and now let's export our network to a bnet file in order to use it with MaBoSS and friends

new_net.write_bnet()

In [ ]:
new_net.generate_sif()

In [ ]:
inter = new_net.interaction_by_label("CDK5", "MAP2K1")

In [ ]:
path = new_net.find_paths(["KRAS", "SRC"])

path

In [ ]:
src = new_net.entity_by_label("LATS1")

In [ ]:
src.info()

In [ ]:
a = new_net.generate_df()

In [ ]:
a